# MARATONA BEHIND THE CODE 2020

## DESAFIO 2: PARTE 2

### Introdução

Na parte 1 deste desafio, você realizou o pré-processamento e o treinamento de um modelo a partir de um conjunto de dados base fornecido. Nesta segunda etapa você irá integrar todas as transformações e eventos de treinamento criados anteriormente em uma Pipeline completa para *deploy* no **Watson Machine Learning**!

### Preparação do Notebook

Primeiro realizaremos a instalação do scikit-learn e a importação das mesmas bibliotecas utilizadas anteriormente

In [1]:
!pip install scikit-learn==0.20.0 --upgrade
!pip install xgboost==0.71 --upgrade

Requirement already up-to-date: scikit-learn==0.20.0 in /opt/conda/envs/Python36/lib/python3.6/site-packages (0.20.0)
     |████████████████████████████████| 501kB 7.3MB/s eta 0:00:01
  Stored in directory: /home/dsxuser/.cache/pip/wheels/4e/6d/1d/0bc23240225fe411315d8abb5d4521b9ff002493ff77515ccc
Successfully built xgboost
  Found existing installation: xgboost 0.72.1
    Uninstalling xgboost-0.72.1:
      Successfully uninstalled xgboost-0.72.1


In [2]:
import json
import requests
import pandas as pd
import numpy as np
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.pipeline import Pipeline
from sklearn.model_selection import KFold, cross_validate
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import ExtraTreesClassifier
import xgboost as xgb

É necessário inserir o conjunto de dados base novamente como um dataframe pandas, seguindo as instruções

![alt text](https://i.imgur.com/K1DwL9I.png "importing-csv-as-df")

Após a seleção da opção **"Insert to code"**, a célula abaixo será preenchida com o código necessário para importação e leitura dos dados no arquivo .csv como um DataFrame Pandas.

In [3]:
import types
import pandas as pd
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share the notebook.
client_adf03a2bfa7046bd889670d7fccdc071 = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='4SV961es8wrewP6RtSh_i74tutu4TqSLqqra8wL5yPUA',
    ibm_auth_endpoint="https://iam.cloud.ibm.com/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url='https://s3-api.us-geo.objectstorage.service.networklayer.com')

body = client_adf03a2bfa7046bd889670d7fccdc071.get_object(Bucket='uninassau-donotdelete-pr-xdmoz1ubhoywsm',Key='dataset_desafio_2.csv')['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

df = pd.read_csv(body)
df.head()


,MATRICULA,NOME,REPROVACOES_DE,REPROVACOES_EM,REPROVACOES_MF,REPROVACOES_GO,NOTA_DE,NOTA_EM,NOTA_MF,NOTA_GO,INGLES,H_AULA_PRES,TAREFAS_ONLINE,FALTAS,PERFIL
0,502375,Márcia Illiglener,0,0,0,0,6.2,5.8,4.6,5.9,0.0,2,4,3,EXATAS
1,397093,Jason Jytereoman Izoimum,0,0,0,0,6.0,6.2,5.2,4.5,1.0,2,4,3,EXATAS
2,915288,Bartolomeu Inácio da Gama,0,0,0,0,7.3,6.7,7.1,7.2,0.0,5,0,3,HUMANAS
3,192652,Fernanda Guedes,1,3,1,1,0.0,0.0,0.0,0.0,1.0,4,4,4,DIFICULDADE
4,949491,Alessandre Borba Gomes,1,3,1,1,0.0,0.0,0.0,0.0,1.0,5,2,5,DIFICULDADE


### Construção da Pipeline completa para encapsulamento no WML

#### Preparando transformações personalizadas para carregamento no WML

Na etapa anterior, foi mostrado como criar uma transformação personalizada, através da declaração de uma classe Python com os métodos ``fit`` e ``transform``.

    - Código da transformação personalizada DropColumns():
    
    from sklearn.base import BaseEstimator, TransformerMixin
    # All sklearn Transforms must have the `transform` and `fit` methods
    class DropColumns(BaseEstimator, TransformerMixin):
        def __init__(self, columns):
            self.columns = columns
        def fit(self, X, y=None):
            return self
        def transform(self, X):
            # Primeiro realizamos a cópia do dataframe 'X' de entrada
            data = X.copy()
            # Retornamos um novo dataframe sem as colunas indesejadas
            return data.drop(labels=self.columns, axis='columns')

Para integrar esses tipos de transformações personalizadas nas Pipelines do Watson Machine Learning, é necessário primeiramente empacotar seu código personalizado como uma biblioteca Python. Isso pode ser feito facilmente com o uso da ferramenta *setuptools*.

No seguinte repositório git: https://github.com/vnderlev/sklearn_transforms temos todos os arquivos necessários para a criação de um pacote Python, nomeado **my_custom_sklearn_transforms**.
Esse pacote possui a seguinte estrutura de arquivos:

    /my_custom_sklearn_transforms.egg-info
        dependency_links.txt
        not-zip-safe
        PKG-INFO
        SOURCES.txt
        top_level.txt
    /my_custom_sklearn_transforms
        __init__.py
        sklearn_transformers.py
    PKG-INFO
    README.md
    setup.cfg
    setup.py
    
O arquivo principal, que irá conter o código das nossas transformadas personalizadas, é o arquivo **/my_custom_sklearn_transforms/sklearn_transformers.py**. Se você acessá-lo no repositório, irá notar que ele contém exatamente o mesmo código declarado na primeira etapa (a classe DropColumns).

Caso você tenha declarado transformações próprias (além da DropColumn fornecida), você deverá adicionar todas as classes dessas transformadas criadas por você nesse mesmo arquivo. Para tal, você deve realizar o fork desse repositório (isso pode ser feito na própria interface Web do Github, clicando no botão conforme a imagem abaixo), e adicionar suas classes personalizadas no arquivo **sklearn_transformers.py**.

![alt text](https://i.imgur.com/D81E1uM.png "forking-a-repo")

Se você somente fez o uso da transformação fornecida (DropColumns), pode ignorar essa etapa de fork, e seguir utilizando o pacote base fornecido! :)

Após a preparação do seu pacote Python com as suas transformadas personalizadas, substitua o link do repositório git na célula abaixo e execute-a. Caso você não tenha preparado nenhuma nova transformada, execute a célula com o link do repositório já fornecido. 

<hr>
    
**OBSERVAÇÃO**

Caso a execução da célula abaixo retorne um erro de que o repositório já existe, execute:

**!rm -r -f sklearn_transforms**

In [1]:
!rm -rf sklearn_transforms
!rm -rf sklearn_transforms.zip

In [2]:
# substitua o link abaixo pelo link do seu repositório git (se for o caso)
!git clone https://github.com/camilabga/sklearn_transforms.git

Cloning into 'sklearn_transforms'...
remote: Enumerating objects: 81, done.
remote: Total 81 (delta 0), reused 0 (delta 0), pack-reused 81
Unpacking objects: 100% (81/81), 13.91 KiB | 111.00 KiB/s, done.


In [3]:
!cd sklearn_transforms
!ls -ltr

total 284
-rwxrwxrwx 1 root root  70573 ago 20 16:21 pipeline.ipynb
drwxrwxrwx 1 root root    352 ago 21 16:11 sklearn_transforms
-rwxrwxrwx 1 root root  70484 ago 21  2020 parte-2.ipynb
-rwxrwxrwx 1 root root 142857 ago 21  2020 parte-1.ipynb


Para subir o código no WML, precisamos enviar um arquivo .zip com todo o código fonte, então iremos zipar o diretório clonado em seguida:

In [4]:
!zip -r sklearn_transforms.zip sklearn_transforms

adding: sklearn_transforms/ (stored 0%)
  adding: sklearn_transforms/.git/ (stored 0%)
  adding: sklearn_transforms/.git/branches/ (stored 0%)
  adding: sklearn_transforms/.git/config (deflated 35%)
  adding: sklearn_transforms/.git/description (deflated 14%)
  adding: sklearn_transforms/.git/HEAD (stored 0%)
  adding: sklearn_transforms/.git/hooks/ (stored 0%)
  adding: sklearn_transforms/.git/hooks/applypatch-msg.sample (deflated 42%)
  adding: sklearn_transforms/.git/hooks/commit-msg.sample (deflated 44%)
  adding: sklearn_transforms/.git/hooks/fsmonitor-watchman.sample (deflated 52%)
  adding: sklearn_transforms/.git/hooks/post-update.sample (deflated 27%)
  adding: sklearn_transforms/.git/hooks/pre-applypatch.sample (deflated 38%)
  adding: sklearn_transforms/.git/hooks/pre-commit.sample (deflated 45%)
  adding: sklearn_transforms/.git/hooks/pre-merge-commit.sample (deflated 39%)
  adding: sklearn_transforms/.git/hooks/pre-push.sample (deflated 50%)
  adding: sklearn_transforms/.g

Com o arquivo zip do nosso pacote carregado no Kernel deste notebook, podemos utilizar a ferramenta pip para instalá-lo, conforme a célula abaixo:

In [5]:
!pip install sklearn_transforms.zip

Processing ./sklearn_transforms.zip
  Created wheel for my-custom-sklearn-transforms: filename=my_custom_sklearn_transforms-1.0-py3-none-any.whl size=2278 sha256=7685da9489d0dfb8d55293a76da0d66b77cf7366a85d78ad18ac3cf57908d547
  Stored in directory: /tmp/pip-ephem-wheel-cache-ng2jfwig/wheels/e0/46/38/8ffacc1b85a68d5ffb89b3ad95c6f3ffd413f2a23b174f30be
Successfully built my-custom-sklearn-transforms
  Attempting uninstall: my-custom-sklearn-transforms
    Found existing installation: my-custom-sklearn-transforms 1.0
    Uninstalling my-custom-sklearn-transforms-1.0:
      Successfully uninstalled my-custom-sklearn-transforms-1.0


Podemos agora realizar a importação do nosso pacote personalizado em nosso notabook!

Iremos importar a transformação DropColumns. Se você possui outras transformações personalizadas, não se esqueça de importá-las!

In [9]:
from my_custom_sklearn_transforms.sklearn_transformers import DropColumns
from my_custom_sklearn_transforms.sklearn_transformers import AdjustMeanSubject
from my_custom_sklearn_transforms.sklearn_transformers import FeatureEngineering

#### Declarando a Pipeline

Após a importação das transformações personalizadas como um pacote Python, podemos partir para a declaração da nossa Pipeline.

O processo é bem semelhante ao realizado na primeira etapa, porém com algumas diferenças importantes, então preste bem atenção!

A Pipeline exemplo possui três estágios: 

    - remover a coluna "NOME"
    - imputar "zeros" em todos os valores faltantes
    - inserir os dados pré-processados como entrada em um modelo treinado
    
Relembrando, a entrada desta Pipeline será o conjunto cru de dados fornecido exceto a coluna "LABELS" (variável-alvo a ser determinada pelo modelo).

Teremos então 17 valores de entrada **na PIPELINE** (no modelo serão 16 entradas, pois a coluna NAME será removida no primeiro estágio após a transformação DropColumn).

    MATRICULA       - número de quatro algarismos único para cada estudante
    NOME            - nome completo do estudante
    FALTAS_DE       - número de faltas na disciplina de ``Direito Empresarial``
    FALTAS_EM       - número de faltas na disciplina de ``Empreendedorismo``
    FALTAS_MF       - número de faltas na disciplina de ``Matemática Financeira``
    MEDIA_DE        - média simples das notas do aluno na disciplina de ``Direito Empresarial`` (0-10)
    MEDIA_EM        - média simples das notas do aluno na disciplina de ``Empreendedorismo`` (0-10)
    MEDIA_MF        - média simples das notas do aluno na disciplina de ``Matemática Financeira`` (0-10)
    HRS_ESTUDO_DE   - horas de estudo particular na disciplina de ``Direito Empresarial``
    HRS_ESTUDO_EM   - horas de estudo particular na disciplina de ``Empreendedorismo``
    HRS_ESTUDO_MF   - horas de estudo particular na disciplina de ``Matemática Financeira``
    REPROVACOES_DE  - número de reprovações na disciplina de ``Direito Empresarial``
    REPROVACOES_EM  - número de reprovações na disciplina de ``Empreendedorismo``
    REPROVACOES_MF  - número de reprovações na disciplina de ``Matemática Financeira``
    LIVROS_TEXTO    - quantidade de livros e textos acessados pelo aluno no sistema da universidade
    AULAS_AO_VIVO   - horas de aulas ao vivo presenciadas pelo aluno (total em todas as disciplinas)
    EXERCICIOS      - número de exercícios realizados pelo estudante (total em todas as disciplinas) no sistema da universidade

A saída da Pipeline será um valor estimado para a coluna "LABELS".

In [10]:
# Criação de uma Transform personalizada ``DropColumns``
rm_columns = DropColumns(
    columns=["NOME","MATRICULA"]
)

In [11]:
adjust_grades = AdjustMeanSubject()

In [12]:
# Criação de um objeto ``SimpleImputer``
si = SimpleImputer(
    missing_values=np.nan,  # os valores faltantes são do tipo ``np.nan`` (padrão Pandas)
    strategy='constant',  # a estratégia escolhida é a alteração do valor faltante por uma constante
    fill_value=1,  # a constante que será usada para preenchimento dos valores faltantes é um int64=0.
    verbose=0,
    copy=True
)

In [13]:
feat_eng = FeatureEngineering()

In [14]:
# Definição das colunas que serão features (nota-se que a coluna NOME não está presente)
features = [
    "MATRICULA", "NOME", 'REPROVACOES_DE', 'REPROVACOES_EM', "REPROVACOES_MF", "REPROVACOES_GO",
    "NOTA_DE", "NOTA_EM", "NOTA_MF", "NOTA_GO",
    "INGLES", "H_AULA_PRES", "TAREFAS_ONLINE", "FALTAS", 
]

# Definição da variável-alvo
target = ["PERFIL"]

# Preparação dos argumentos para os métodos da biblioteca ``scikit-learn``
X = df[features]
y = df[target]

**ATENÇÃO!!**

A célula acima, embora muito parecida com a definição de features na primeira etapa deste desafio, possui uma grande diferença!

Nela está presente a coluna "NOME" como uma feature! Isso ocorre pois neste caso essas são as entradas da *PIPELINE*, e não do modelo.

In [15]:
# Separação dos dados em um conjunto de treino e um conjunto de teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=337)

Na célula abaixo é realizada a declaração de um objeto **Pipeline** do scikit-learn, onde é declarado o parâmetro *steps*, que nada mais é do que uma lista com as etapas da nossa pipeline:

    'remove_cols'     - transformação personalizada DropColumns
    'imputer'         - transformação embutida do scikit-learn para imputação de valores faltantes
    'dtc'             - um classificador via árvore de decisão
    
Note que passamos como passos as transformadas instanciadas anteriormente, sob nome `rm_columns` e `si`.

In [16]:
# Criação da nossa pipeline para armazenamento no Watson Machine Learning:
my_pipeline = Pipeline(
    steps=[
        ('remove_cols', rm_columns),
        ('adjust_grades', adjust_grades),
        ('feat_engineering', feat_eng),
        ('imputer', si),
        ('scaler', StandardScaler()),
        ('clf', xgb.XGBClassifier(learning_rate=0.005,  
                      n_estimators=1000, 
                      max_depth=50, 
                      gamma=10)),
         #ExtraTreesClassifier(n_estimators=1000, random_state=0)),
    ]
)

Em seguida iremos executar o método `fit()` da Pipeline, realizando o pré-processamento e o treinamento do modelo de uma só vez.

In [17]:
# Inicialização da Pipeline (pré-processamento e realização do treinamento do modelo)
my_pipeline.fit(X_train, y_train)

/opt/conda/envs/Python36/lib/python3.6/site-packages/sklearn/preprocessing/label.py:219: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/opt/conda/envs/Python36/lib/python3.6/site-packages/sklearn/preprocessing/label.py:252: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Pipeline(memory=None,
     steps=[('remove_cols', DropColumns(columns=['NOME', 'MATRICULA'])), ('adjust_grades', AdjustMeanSubject()), ('feat_engineering', FeatureEngineering()), ('imputer', SimpleImputer(copy=True, fill_value=1, missing_values=nan,
       strategy='constant', verbose=0)), ('scaler', StandardScaler(copy=True,...
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=1))])

Agora que temos uma pipeline completa, com etapas de pré-processamento configuradas e também um modelo por árvore de decisão já treinado, podemos realizar a integração com o Watson Machine Learning!

### Encapsulando uma Pipeline personalizada no Watson Machine Learning

#### Estabelecendo conexão entre o cliente Python do WML e a sua instância do serviço na nuvem

In [18]:
# Biblioteca Python com implementação de um cliente HTTP para a API do WML
from watson_machine_learning_client import WatsonMachineLearningAPIClient

As próximas células irão realizar o deploy da pipeline declarada neste notebook no WML. Só prossiga se você já está satisfeito com seu modelo e acha que já é a hora de fazer o deploy da sua solução.

Cole as credenciais de sua instância do Watson Machine Learning na variável na célula abaixo.

É importante que a variável que contém os valores tenha o nome de ``wml_credentials`` para que as próximas células deste notebook executem corretamente.

In [19]:
wml_credentials = {
  "apikey": "24-p3uYtxnYf6Qctt0-kcy0qES4PwaHUvLjycOQd3iUt",
  "iam_apikey_description": "Auto-generated for key dd79d61d-903d-4757-8532-e8777a5255ca",
  "iam_apikey_name": "Service credentials-1",
  "iam_role_crn": "crn:v1:bluemix:public:iam::::serviceRole:Writer",
  "iam_serviceid_crn": "crn:v1:bluemix:public:iam-identity::a/1e8f8b5e7e9c42c0b0f186c862ee3b3f::serviceid:ServiceId-928352af-9fa1-4b06-9ae8-6e7acb86b4f6",
  "instance_id": "708a5d14-c71f-4f69-ac6f-aac79eb7ecf2",
  "url": "https://eu-de.ml.cloud.ibm.com"
}

In [20]:
# Instanciando um objeto cliente do Watson Machine Learning a partir das credenciais fornecidas

clientWML = WatsonMachineLearningAPIClient(wml_credentials)

In [21]:
# Extraindo detalhes da sua instância do Watson Machine Learning
instance_details = clientWML.service_instance.get_details()
print(json.dumps(instance_details, indent=4))

{
    "entity": {
        "source": "Bluemix",
        "published_models": {
            "url": "https://eu-de.ml.cloud.ibm.com/v3/wml_instances/708a5d14-c71f-4f69-ac6f-aac79eb7ecf2/published_models"
        },
        "usage": {
            "capacity_units": {
                "current": 0,
                "limit": 180000000
            },
            "computation_time": {
                "current": 0,
                "limit": 180000
            },
            "deployment_count": {
                "current": 2,
                "limit": 5
            },
            "expiration_date": "2020-09-01T00:00:00.000Z",
            "gpu_count_k80": {
                "current": 0,
                "limit": 8
            },
            "gpu_count_p100": {
                "current": 0,
                "limit": 0
            },
            "gpu_count_v100": {
                "current": 0,
                "limit": 0
            },
            "model_count": {
                "current": 5,
            

**ATENÇÃO!!**

Fique atento para os limites de consumo de sua instância do Watson Machine Learning!

Caso você expire a camada grátis, não será possível avaliar seu modelo (pois é necessária a realização de algumas chamadas de API que consomem predições!)

#### Listando todos os artefatos armazenados no seu WML

Para listar todos os artefatos armazenados em seu Watson Machine Learning, você pode usar a seguinte função:

    clientWML.repository.list()

In [22]:
# Listando todos os artefatos atualmente armazenados na sua instância do WML

clientWML.repository.list()

------------------------------------  -------------------------------  ------------------------  -----------------  -----------------
GUID                                  NAME                             CREATED                   FRAMEWORK          TYPE
f3940d15-e493-4ddb-809e-4499a6379613  desafio-2-mbtc2020-pipeline-5    2020-08-21T17:50:54.462Z  scikit-learn-0.20  model
3fbc65a7-843e-43dd-a2a6-7fd41422dd8a  desafio-2-mbtc2020-pipeline-4    2020-08-21T16:15:25.798Z  scikit-learn-0.20  model
12ddf8aa-6e8a-460f-b752-6cfd027bf89c  desafio-2-mbtc2020-pipeline-3    2020-08-21T16:07:42.747Z  scikit-learn-0.20  model
4f009f7b-efd6-401b-9944-181faaac2322  desafio-2-mbtc2020-pipeline-2    2020-08-21T15:43:36.449Z  scikit-learn-0.20  model
006b6f7b-5e8e-454f-83a4-412f1edba41d  desafio-2-mbtc2020-pipeline-1    2020-08-21T15:32:59.281Z  scikit-learn-0.20  model
9d04fa6f-8df3-4252-9707-03f7e04d5b20  desafio-2-mbtc2020-deployment-4  2020-08-21T16:23:34.858Z  scikit-learn-0.20  online deployment
f

No plano LITE do Watson Machine Learning só é permitido o deploy de um único modelo por vez. Se for o caso de você já possuir um modelo online na sua instância, você pode apagá-lo utilizando o método clientWML.repository.delete():

    artifact_guid = "359c8951-d2fe-4063-8706-cc06b32d5e0d"
    clientWML.repository.delete(artifact_guid)

#### Criando uma nova definição de pacote Python personalizado no WML

O primeiro passo para realizar seu deploy é armazenar o código das transformações personalizadas criadas por você.

Para essa etapa precisamos apenas do arquivo .zip do pacote criado (que já possuimos carregado no Kernel!)

In [23]:
# Definição de metadados do nosso pacote com as Transforms personalizadas
pkg_meta = {
    clientWML.runtimes.LibraryMetaNames.NAME: "my_custom_sklearn_transform_6",
    clientWML.runtimes.LibraryMetaNames.DESCRIPTION: "A custom sklearn transform",
    clientWML.runtimes.LibraryMetaNames.FILEPATH: "sklearn_transforms.zip",  # Note que estamos utilizando o .zip criado anteriormente!
    clientWML.runtimes.LibraryMetaNames.VERSION: "1.0",
    clientWML.runtimes.LibraryMetaNames.PLATFORM: { "name": "python", "versions": ["3.6"] }
}
custom_package_details = clientWML.runtimes.store_library( pkg_meta )
custom_package_uid = clientWML.runtimes.get_library_uid( custom_package_details )

print("\n Lista de artefatos de runtime armazenados no WML:")
clientWML.repository.list()


 Lista de artefatos de runtime armazenados no WML:
------------------------------------  -------------------------------  ------------------------  -----------------  -----------------
GUID                                  NAME                             CREATED                   FRAMEWORK          TYPE
f3940d15-e493-4ddb-809e-4499a6379613  desafio-2-mbtc2020-pipeline-5    2020-08-21T17:50:54.462Z  scikit-learn-0.20  model
3fbc65a7-843e-43dd-a2a6-7fd41422dd8a  desafio-2-mbtc2020-pipeline-4    2020-08-21T16:15:25.798Z  scikit-learn-0.20  model
12ddf8aa-6e8a-460f-b752-6cfd027bf89c  desafio-2-mbtc2020-pipeline-3    2020-08-21T16:07:42.747Z  scikit-learn-0.20  model
4f009f7b-efd6-401b-9944-181faaac2322  desafio-2-mbtc2020-pipeline-2    2020-08-21T15:43:36.449Z  scikit-learn-0.20  model
006b6f7b-5e8e-454f-83a4-412f1edba41d  desafio-2-mbtc2020-pipeline-1    2020-08-21T15:32:59.281Z  scikit-learn-0.20  model
9d04fa6f-8df3-4252-9707-03f7e04d5b20  desafio-2-mbtc2020-deployment-4  2020-08-21T1

#### Criando uma nova definição de runtime Python personalizado no WML

O segundo passo é armazenar uma definição de runtime Python para utilizar a nossa biblioteca personalizada.

Isso pode ser feito da seguinte forma:

In [24]:
runtime_meta = {
    clientWML.runtimes.ConfigurationMetaNames.NAME: "my_custom_wml_runtime_6",
    clientWML.runtimes.ConfigurationMetaNames.DESCRIPTION: "A Python runtime with custom sklearn Transforms",
    clientWML.runtimes.ConfigurationMetaNames.PLATFORM: {
        "name": "python",
        "version": "3.6"
    },
    clientWML.runtimes.ConfigurationMetaNames.LIBRARIES_UIDS: [ custom_package_uid ]
}
runtime_details = clientWML.runtimes.store( runtime_meta )
custom_runtime_uid = clientWML.runtimes.get_uid( runtime_details )

print("\n Detalhes do runtime armazenado:")
print(json.dumps(runtime_details, indent=4))


 Detalhes do runtime armazenado:
{
    "entity": {
        "name": "my_custom_wml_runtime_6",
        "description": "A Python runtime with custom sklearn Transforms",
        "custom_libraries": [
            {
                "name": "my_custom_sklearn_transform_6",
                "url": "https://private.eu-de.ml.cloud.ibm.com/v4/libraries/47f8bcde-8baa-4e10-9fc6-82eec5682288",
                "version": "1.0"
            }
        ],
        "content_url": "https://private.eu-de.ml.cloud.ibm.com/v4/runtimes/ecb1ca52-4d28-4a7d-840e-9ea814545d7b/content",
        "platform": {
            "name": "python",
            "version": "3.6"
        }
    },
    "metadata": {
        "created_at": "2020-08-21T18:09:45.938Z",
        "guid": "ecb1ca52-4d28-4a7d-840e-9ea814545d7b",
        "url": "https://eu-de.ml.cloud.ibm.com/v4/runtimes/ecb1ca52-4d28-4a7d-840e-9ea814545d7b"
    }
}


In [25]:
# Listando todos runtimes armazenados no seu WML:
clientWML.runtimes.list()

------------------------------------  -----------------------  ------------------------  ----------
GUID                                  NAME                     CREATED                   PLATFORM
ecb1ca52-4d28-4a7d-840e-9ea814545d7b  my_custom_wml_runtime_6  2020-08-21T18:09:45.938Z  python-3.6
9823618d-f650-4f83-8ae6-9c514dfb0511  my_custom_wml_runtime_5  2020-08-21T17:50:45.429Z  python-3.6
976610f8-7509-4e44-908f-1db8f9d270bd  my_custom_wml_runtime_4  2020-08-21T16:15:23.264Z  python-3.6
7c423399-582d-47a0-831c-6d175c5d0b09  my_custom_wml_runtime_3  2020-08-21T16:07:40.225Z  python-3.6
b96d386d-247b-4012-aa95-d8d5e61a3b60  my_custom_wml_runtime_2  2020-08-21T15:43:18.814Z  python-3.6
470220f3-304b-430f-b7cb-84ebe55a1e21  my_custom_wml_runtime_1  2020-08-21T15:31:50.352Z  python-3.6
------------------------------------  -----------------------  ------------------------  ----------


#### Criando uma nova definição de Pipeline personalizada no WML

Finalmente iremos criar uma definição (metadados) para a nossa Pipeline ser hospedada no WML.

Definimos como parâmetros um nome para o artefato e o ID do runtime criado anteriormente.

In [26]:
model_meta = {
    clientWML.repository.ModelMetaNames.NAME: 'desafio-2-mbtc2020-pipeline-6',
    clientWML.repository.ModelMetaNames.DESCRIPTION: "my pipeline for submission",
    clientWML.repository.ModelMetaNames.RUNTIME_UID: custom_runtime_uid
}

Em seguida chamamos o método para armazenar a nova definição:

In [27]:
# Função para armazenar uma definição de Pipeline no WML
stored_model_details = clientWML.repository.store_model(
    model=my_pipeline,  # `my_pipeline` é a variável criada anteriormente e contém nossa Pipeline já treinada :)
    meta_props=model_meta,  # Metadados definidos na célula anterior
    training_data=None  # Não altere esse parâmetro
)

print("\n Lista de artefatos armazenados no WML:")
clientWML.repository.list()

# Detalhes do modelo hospedado no Watson Machine Learning
print("\n Metadados do modelo armazenado:")
print(json.dumps(stored_model_details, indent=4))


 Lista de artefatos armazenados no WML:
------------------------------------  -------------------------------  ------------------------  -----------------  -----------------
GUID                                  NAME                             CREATED                   FRAMEWORK          TYPE
7626d512-08e8-4d1e-9e63-3b2e377af7a7  desafio-2-mbtc2020-pipeline-6    2020-08-21T18:09:48.501Z  scikit-learn-0.20  model
f3940d15-e493-4ddb-809e-4499a6379613  desafio-2-mbtc2020-pipeline-5    2020-08-21T17:50:54.462Z  scikit-learn-0.20  model
3fbc65a7-843e-43dd-a2a6-7fd41422dd8a  desafio-2-mbtc2020-pipeline-4    2020-08-21T16:15:25.798Z  scikit-learn-0.20  model
12ddf8aa-6e8a-460f-b752-6cfd027bf89c  desafio-2-mbtc2020-pipeline-3    2020-08-21T16:07:42.747Z  scikit-learn-0.20  model
4f009f7b-efd6-401b-9944-181faaac2322  desafio-2-mbtc2020-pipeline-2    2020-08-21T15:43:36.449Z  scikit-learn-0.20  model
006b6f7b-5e8e-454f-83a4-412f1edba41d  desafio-2-mbtc2020-pipeline-1    2020-08-21T15:32:59.281

#### Realizando o deployment do seu modelo para consumo imediato por outras aplicações

In [28]:
# O deployment do modelo é finalmente realizado por meio do método ``deployments.create()``

model_deployment_details = clientWML.deployments.create(
    artifact_uid=stored_model_details["metadata"]["guid"],  # Não altere esse parâmetro
    name="desafio-2-mbtc2020-deployment-6",
    description="Solução do desafio 2 - MBTC",
    asynchronous=False,  # Não altere esse parâmetro
    deployment_type='online',  # Não altere esse parâmetro
    deployment_format='Core ML',  # Não altere esse parâmetro
    meta_props=model_meta  # Não altere esse parâmetro
)



#######################################################################################

Synchronous deployment creation for uid: '7626d512-08e8-4d1e-9e63-3b2e377af7a7' started

#######################################################################################


INITIALIZING
DEPLOY_IN_PROGRESS...
DEPLOY_SUCCESS


------------------------------------------------------------------------------------------------
Successfully finished deployment creation, deployment_uid='2e9436af-9a70-48cc-82ee-3c01c3413102'
------------------------------------------------------------------------------------------------




#### Testando um modelo hospedado no Watson Machine Learning

In [29]:
# Recuperando a URL endpoint do modelo hospedado na célula anterior

model_endpoint_url = clientWML.deployments.get_scoring_url(model_deployment_details)
print("A URL de chamada da sua API é: {}".format(model_endpoint_url))

A URL de chamada da sua API é: https://eu-de.ml.cloud.ibm.com/v3/wml_instances/708a5d14-c71f-4f69-ac6f-aac79eb7ecf2/deployments/2e9436af-9a70-48cc-82ee-3c01c3413102/online


In [30]:
# Detalhes do deployment realizado

deployment_details = clientWML.deployments.get_details(
    deployment_uid=model_deployment_details["metadata"]["guid"]  # esse é o ID do seu deployment!
)

print("Metadados do deployment realizado: \n")
print(json.dumps(deployment_details, indent=4))

Metadados do deployment realizado: 

{
    "metadata": {
        "guid": "2e9436af-9a70-48cc-82ee-3c01c3413102",
        "url": "https://eu-de.ml.cloud.ibm.com/v3/wml_instances/708a5d14-c71f-4f69-ac6f-aac79eb7ecf2/deployments/2e9436af-9a70-48cc-82ee-3c01c3413102",
        "created_at": "2020-08-21T18:11:32.412Z",
        "modified_at": "2020-08-21T18:11:32.943Z"
    },
    "entity": {
        "runtime_environment": "python-3.6",
        "name": "desafio-2-mbtc2020-deployment-6",
        "scoring_url": "https://eu-de.ml.cloud.ibm.com/v3/wml_instances/708a5d14-c71f-4f69-ac6f-aac79eb7ecf2/deployments/2e9436af-9a70-48cc-82ee-3c01c3413102/online",
        "deployable_asset": {
            "name": "desafio-2-mbtc2020-pipeline-6",
            "url": "https://eu-de.ml.cloud.ibm.com/v3/wml_instances/708a5d14-c71f-4f69-ac6f-aac79eb7ecf2/published_models/7626d512-08e8-4d1e-9e63-3b2e377af7a7",
            "guid": "7626d512-08e8-4d1e-9e63-3b2e377af7a7",
            "description": "my pipeline for s

In [31]:
scoring_payload = {
    'fields': [
        "MATRICULA", "NOME", 'REPROVACOES_DE', 'REPROVACOES_EM', "REPROVACOES_MF", "REPROVACOES_GO",
        "NOTA_DE", "NOTA_EM", "NOTA_MF", "NOTA_GO",
        "INGLES", "H_AULA_PRES", "TAREFAS_ONLINE", "FALTAS", 
    ],
    'values': [
        [
            513949,"Marli Quésia de Oliveira",1,1,1,1,4.3,4.0,3.1,4.9,0,3,4,3,
        ]
    ]
}

print("\n Payload de dados a ser classificada:")
print(json.dumps(scoring_payload, indent=4))


 Payload de dados a ser classificada:
{
    "fields": [
        "MATRICULA",
        "NOME",
        "REPROVACOES_DE",
        "REPROVACOES_EM",
        "REPROVACOES_MF",
        "REPROVACOES_GO",
        "NOTA_DE",
        "NOTA_EM",
        "NOTA_MF",
        "NOTA_GO",
        "INGLES",
        "H_AULA_PRES",
        "TAREFAS_ONLINE",
        "FALTAS"
    ],
    "values": [
        [
            513949,
            "Marli Qu\u00e9sia de Oliveira",
            1,
            1,
            1,
            1,
            4.3,
            4.0,
            3.1,
            4.9,
            0,
            3,
            4,
            3
        ]
    ]
}


In [32]:
result = clientWML.deployments.score(
    model_endpoint_url,
    scoring_payload
)

print("\n Resultados:")
print(json.dumps(result, indent=4))


 Resultados:
{
    "fields": [
        "prediction",
        "probability"
    ],
    "values": [
        [
            "DIFICULDADE",
            [
                0.9766823053359985,
                0.012689008377492428,
                0.003492891788482666,
                0.0031599223148077726,
                0.0039759366773068905
            ]
        ]
    ]
}


<hr>

## Parabéns! 

Se tudo foi executado sem erros, você já tem um classificador baseado em machine learning encapsulado como uma API REST!

Para testar a sua solução integrada com um assistente virtual e realizar a submissão, acesse a página:

https://uninassau.maratona.dev

Você irá precisar da endpoint url do seu modelo e das credenciais do WML :)